In [38]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [39]:
# need to define an input size for padding
input_size = [30, 38, 37, 1]

In [40]:
def apply_padding(arr, final_shape, fill_value=0):
    """ See Padding.__call__().
    """
    orig_shape = arr.shape
    padding = []
    # print(f"SHAPES: {orig_shape} - {self.shape}")
    for orig_i, final_i in zip(orig_shape, final_shape):
        shape_i = final_i - orig_i
        half_shape_i = shape_i // 2
        if shape_i % 2 == 0:
            padding.append((half_shape_i, half_shape_i))
        else:
            padding.append((half_shape_i, half_shape_i + 1))
    for cnt in range(len(arr.shape) - len(padding)):
        padding.append((0, 0))

    fill_arr = np.pad(arr, padding, mode="constant",
                        constant_values=fill_value)
    return(fill_arr)

In [41]:
crops_dir = '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/'
#regions_list = ['S.Or.', 'CINGULATE', 'F.I.P.', 'S.T.s.', 'S.C.-sylv.']
regions_list = ['S.Or.', 'CINGULATE']
sides = ['L', 'R']

In [42]:
save_dir = os.path.join(crops_dir, f'multiple_regions/mask/')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [43]:
for datatype, fill_value in zip(['skeleton', 'label', 'distbottom'], [0, 0, 32500]):
    skels_list = []
    subject_list = []
    for sulcus, side in zip(regions_list, sides):
        # stack skels
        skels = np.load(os.path.join(crops_dir, f'{sulcus}/mask/{side}{datatype}.npy'))
        print(sulcus, skels.shape)
        shape = [skels.shape[0]] + input_size
        skels = apply_padding(skels, shape, fill_value=fill_value)
        skels_list.append(skels)
        # stack subjects
        subs = pd.read_csv(os.path.join(crops_dir, f'{sulcus}/mask/{side}{datatype}_subject.csv'))
        l = subs['Subject'].tolist()
        l = [sulcus+'_'+elem for elem in l]
        subject_list+=l
    skels = np.vstack(skels_list)
    np.save(save_dir + f'{datatype}.npy', skels)
    subs = pd.DataFrame({'Subject': subject_list})
    subs.to_csv(save_dir + f'{datatype}_subject.csv', index=False)

S.Or. (21051, 26, 34, 23, 1)
CINGULATE (21052, 16, 37, 37, 1)
S.Or. (21051, 26, 34, 23, 1)
CINGULATE (21052, 16, 37, 37, 1)
S.Or. (21051, 26, 34, 23, 1)
CINGULATE (21052, 16, 37, 37, 1)


In [44]:
# train_val_subjects required
subs.to_csv(save_dir + f'train_val_subjects.csv', index=False, header=None)